In [1]:
import os, sys, email, datetime, pprint, re, time, html, pickle
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [2]:
### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")
from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

In [4]:
from db import connect
engine = connect()
mail_df = pd.read_sql(sql='SELECT DISTINCT id, body FROM mail',con=engine, index_col=None)

In [6]:
test = mail_df[mail_df['id']=='4084867E.9050502@kbc.net.au'].values

In [7]:
test

array([['4084867E.9050502@kbc.net.au',
        'Jesper Tverskov writes:\n One of the recommendations from City University on the WAI Guidelines said:\n - improve search design \n<content trimmed/>\n Now, from the point of view of accessibility, when a website\'s pages are indexed by Google, wouldn\'t it be easier for people with disabilities to use a seach engine they are already used to use? Should we actively promote the use of search engines like Google as local seach engine at any major website, if the pages are already indexed by Google?\nMatthew Smith replies:\nWhilst I think that Google has a great product, I do not think it in the best \ninterests of accessibility, or indeed of a stable infrastructure, to standardise \non a single, proprietary solution.\nIf Google should fail as a company or start charging excessively for the Site \nSearch service), if everyone were using it, the entire concept of Site Search \nwould drop into a vacuum.\nWere Google an Open Source project, with

In [26]:
for match in re.finditer('Google',test[0][1]):
    print(match.span())

(231, 237)
(400, 406)
(487, 493)
(538, 544)
(717, 723)
(903, 909)
(990, 996)
(1063, 1069)
(1152, 1158)


In [32]:
# MinIEにかける
import requests
import json

pattern = re.compile('\W+')
count = 0
openie = {}

for value in test:
    uid = value[0]
    body = value[1]
    doc = nlp(body)
    result_list = []
    not_sent_count = 0

    for sent in doc.sents:
        sent_text = sent.text
        if sent_text.isnumeric() or pattern.fullmatch(sent_text) or len(sent_text.split()) <= 5:
            not_sent_count = not_sent_count + 1
            pass
        else:
            match = re.search(re.escape(sent_text), doc.text)
            offset_info = {'sentence':sent_text, 'start':match.start(), 'end':match.end()}
            try:
                response = requests.post('http://localhost:8080/minie/query', data=sent.text.encode('utf-8'))
                result = response.json()
                if result['facts'] != []:
                    offset_info.update(result)
                    result_list.append(offset_info)
            except json.JSONDecodeError:
                pass
    
    openie[uid] = result_list

In [68]:
with open('/Users/taroaso/myprojects/OpenIE/trec/output/tagme.pickle', mode="rb") as f:
    tagme_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/ner.pickle', mode="rb") as f:
    ner_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/type_translation.pickle', mode="rb") as f:
    translation = pickle.load(f)

In [96]:
oie_triples = []
# メールごとにopenIEの結果を取り出す
for uid, oie_results in openie.items():
    # openIEの結果とエンティティの抽出結果があるときのみ,処理を継続する
    if oie_results == [] or tagme_result[uid] is None:
        pass
    else:
        # メールの文ごとにopenIEの結果を取り出す
        for result in oie_results:
            sentence = result['sentence']
            sent_start = result['start']
            facts = result['facts']
            # トリプルを取り出す
            for fact in facts:
                s = fact['subject']
                o = fact['object']
                # subjectとobjectのオフセットを取得する
                s_match = re.search(re.escape(s),sentence)
                o_match = re.search(re.escape(o),sentence)
                if s_match is None:
                    pass
                else:
                    s_start = s_match.start() + sent_start
                    s_end = s_match.end() + sent_start
                    # tagmeのアノテーション結果を走査する
                    for ann in tagme_result[uid]['annotations']:         
                        # トリプルの主語とspot, start, endが一致する場合，トリプル（主語，メンション，検出されたスポットのリソース）作成
                        if s == ann['spot'] and s_start == ann['start'] and s_end == ann['end'] and ann['link_probability'] >= 0.3:
                            emailmessage = uid
                            offset_based_string = emailmessage + '#offset_' + str(s_start) + '_' + str(s_end)
                            oie_triples.append((s,'mentions',offset_based_string))
                            if o_match is None:
                                oie_triples.append((s,fact['predicate'],o))
                            else:
                                o_start = o_match.start() + sent_start
                                o_end = o_match.end() + sent_start
                                # tagmeのアノテーション結果を走査する
                                for ann in tagme_result[uid]['annotations']:
                                    #　トリプルの目的語と一致するspotがある場合，（目的語，メンション，検出されたスポットのリソース）作成
                                    if o == ann['spot'] and o_start == ann['start'] and o_end == ann['end'] and ann['link_probability'] >= 0.3:
                                        offset_based_string = emailmessage + '#offset_' + str(o_start) + '_' + str(o_end)
                                        oie_triples.append((o,'mentions',offset_based_string))
                                        #openIEのトリプル作成
                                        oie_triples.append((s,fact['predicate'],o))
                                oie_triples.append((s,fact['predicate'],o))

o is match
o is match
o is match
o is match
o is match


In [97]:
oie_triples

[('Google', 'mentions', '4084867E.9050502@kbc.net.au#offset_400_406'),
 ('Google', 'is', 'search engine'),
 ('Google', 'mentions', '4084867E.9050502@kbc.net.au#offset_538_544'),
 ('Google', 'has', 'great product'),
 ('Google', 'mentions', '4084867E.9050502@kbc.net.au#offset_538_544'),
 ('Google', 'has', 'great product'),
 ('Google', 'mentions', '4084867E.9050502@kbc.net.au#offset_717_723'),
 ('Google', 'fail as', 'company'),
 ('Google', 'mentions', '4084867E.9050502@kbc.net.au#offset_717_723'),
 ('Google', 'start', 'charging excessively')]

In [78]:
openie

{'4084867E.9050502@kbc.net.au': [{'sentence': "Now, from the point of view of accessibility, when a website's pages are indexed by Google, wouldn't it be easier for people with disabilities to use a seach engine they are already used to use?",
   'start': 147,
   'end': 341,
   'facts': [{'subject': 'website', 'predicate': 'has', 'object': 'pages'},
    {'subject': "website 's pages",
     'predicate': 'are indexed by',
     'object': 'Google'},
    {'subject': 'it', 'predicate': 'be', 'object': 'easier'},
    {'subject': 'it',
     'predicate': 'be easier from',
     'object': 'point of view of accessibility'},
    {'subject': 'it',
     'predicate': 'be easier for',
     'object': 'people with disabilities to use seach engine'},
    {'subject': 'disabilities',
     'predicate': 'be use',
     'object': 'seach engine'},
    {'subject': 'they',
     'predicate': 'are used to use',
     'object': 'seach engine already'},
    {'subject': 'they',
     'predicate': 'are used to use',
     

In [46]:
tagme_result['4084867E.9050502@kbc.net.au']['annotations']

[{'spot': 'Tverskov',
  'start': 7,
  'link_probability': 1,
  'rho': 0.5,
  'end': 15,
  'id': 50023390,
  'title': 'Jeppe Tverskov'},
 {'spot': 'One',
  'start': 25,
  'link_probability': 0.0015941697638481855,
  'rho': 0.020603865385055542,
  'end': 28,
  'id': 995724,
  'title': 'One (Metallica song)'},
 {'spot': 'recommendations',
  'start': 36,
  'link_probability': 0.001484526670537889,
  'rho': 0.0941823273897171,
  'end': 51,
  'id': 596646,
  'title': 'Recommender system'},
 {'spot': 'City',
  'start': 57,
  'link_probability': 0.011296111159026623,
  'end': 61},
 {'spot': 'City University',
  'start': 57,
  'link_probability': 0.04440648853778839,
  'rho': 0.05870533734560013,
  'end': 72,
  'id': 533484,
  'title': 'City University London'},
 {'spot': 'University',
  'start': 62,
  'link_probability': 0.0077834646217525005,
  'end': 72},
 {'spot': 'WAI',
  'start': 80,
  'link_probability': 0.05882352963089943,
  'rho': 0.05991806089878082,
  'end': 83,
  'id': 181070,
  't